In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import glob

import tifffile as tf

import os

path = "/home/patrick/github/data"

filenames = glob.glob(os.path.join(path, "dm*/**/*.tif*"), recursive=True)

print(len(filenames))

In [ ]:
# filter lamella, landing

print(len(filenames))
# filenames = [fname for fname in filenames if "ref_landing" not in fname and "ref_lamella" not in fname and "trench" not in fname and "low_res" not in fname]

filenames = [fname for fname in filenames if "needle" in fname]
print(len(filenames))

In [ ]:
for fname in filenames[:10]:
    img = tf.imread(fname)
    plt.imshow(img, cmap="gray")
    plt.show()

In [ ]:
save_path = "/home/patrick/github/data/training"
for i, fname in enumerate(filenames):
    
    img = tf.imread(fname)
    new_fname = os.path.join(save_path,f"a{i:05d}.tif")

    tf.imsave(new_fname, img)


In [ ]:
train_filenames = sorted(glob.glob(os.path.join(save_path, "*.tif")))
for fname in train_filenames[:10]:

    img = tf.imread(fname)
    plt.imshow(img, cmap="gray")
    plt.show()

In [ ]:
import zarr

images = zarr.open(tf.imread(os.path.join("/home/patrick/github/data/training/", "images", "*.tif*"), aszarr=True)) 
labels = zarr.open(tf.imread(os.path.join("/home/patrick/github/data/training/", "labels","*.tif*"), aszarr=True))


print(images.shape)
print(labels.shape)


for img, label in zip(images, labels):


    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    ax[0].imshow(img, cmap="gray")
    ax[1].imshow(label)
    plt.show()



### Model Testing

In [ ]:
%load_ext autoreload
%autoreload 2 


import glob
import os
import tifffile as tf
import matplotlib.pyplot as plt

images = sorted(glob.glob(os.path.join("/home/patrick/github/data/training/train/images", "*.tif*")))
labels = sorted(glob.glob(os.path.join("/home/patrick/github/data/training/train/labels", "*.tif*")))

for ii, (i, l) in enumerate(zip(images, labels)):
    print(os.path.basename(i), os.path.basename(l))

    img, lbl = tf.imread(i), tf.imread(l)

    fig, ax = plt.subplots(1, 2, figsize=(7, 5))
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Image")
    ax[1].imshow(lbl)
    ax[1].set_title("Label")
    plt.show()
    
    if ii == 2:
        break



In [ ]:
filenames = sorted(glob.glob(os.path.join("/home/patrick/github/data/liftout/training/train/images/", "*.tif*")))

print("files: ", len(filenames))
# filenames.append(*list(sorted(glob.glob(os.path.join("/home/patrick/github/data/training/train/images/", "*.tif*")))
print("files: ", len(filenames))
import random
random.shuffle(filenames)


from fibsem.segmentation.model import SegmentationModel

baseline = "/home/patrick/github/fibsem/fibsem/segmentation/models/model.pt"
# checkpoint_2 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_01_51_n08_model.pt"
# checkpoint_5 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_00_58_n05_model.pt"
# checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/03_11_2022_19_02_27_n10_model.pt"
# checkpoints = [baseline, checkpoint_2, checkpoint_5, checkpoint_10]

small_model = "/home/patrick/github/fibsem/fibsem/segmentation/models/model2.pt"
# checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_00_10_n10_model.pt"
# checkpoint_15 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_09_22_n15_model.pt"
# checkpoint_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined/03_11_2022_20_18_32_n20_model.pt"

checkpoint_10 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_37_32_n10_model.pt"
checkpoint_15 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_46_48_n15_model.pt"
checkpoint_20 = "/home/patrick/github/fibsem/fibsem/segmentation/models/combined34/04_11_2022_18_56_00_n20_model.pt"


checkpoints = [baseline, small_model, checkpoint_10, checkpoint_15, checkpoint_20]
for i, fname in enumerate(filenames):

    img = tf.imread(fname)

    print(i, fname) 

    fig, ax = plt.subplots(1, len(checkpoints) + 1, figsize=(15, 5))
    ax[0].imshow(img, cmap="gray")
    ax[0].set_title("Image")


    titles = ["Baseline", "Update", "Checkpoint 10", "Checkpoint 15", "Checkpoint 20"]
    for j, (checkpoint, title) in enumerate(zip(checkpoints, titles), 1):
        model = SegmentationModel(checkpoint=checkpoint, num_classes=3)

        mask = model.inference(img)

        ax[j].imshow(mask)
        ax[j].set_title(title)

    save_path = "results34"
    os.makedirs(save_path, exist_ok=True)
    fig.savefig(os.path.join(save_path, f"{os.path.basename(fname).replace('tif', 'png')}"))

    plt.show()

### Pattern Testing


In [1]:
%load_ext autoreload
%autoreload 2

from fibsem import utils, patterning
import matplotlib.pyplot as plt

# protocol_path = "/home/patrick/github/autoliftout/liftout/protocol/protocol.yaml"
protocol_path = r"C:\Users\pcle0002\Documents\repos\autoliftout\liftout\protocol\protocol.yaml"
microscope, settings = utils.setup_session(protocol_path=protocol_path)

2023-03-12 21:23:07,204 — root — INFO — connect_to_microscope:3125 — Connected to Demo Microscope
2023-03-12 21:23:07,206 — root — INFO — setup_session:227 — Finished setup for session: autoliftout_2023-03-12-09-23-07PM


In [3]:
patterns_fn = {f"{pattern.name}": pattern for pattern in patterning.__PATTERNS__}

from pprint import pprint

pprint(patterns_fn)

{'Circle': <class 'fibsem.patterning.CirclePattern'>,
 'Fiducial': <class 'fibsem.patterning.FiducialPattern'>,
 'Horseshoe': <class 'fibsem.patterning.HorseshoePattern'>,
 'Line': <class 'fibsem.patterning.LinePattern'>,
 'MicroExpansion': <class 'fibsem.patterning.MicroExpansionPattern'>,
 'Rectangle': <class 'fibsem.patterning.RectanglePattern'>,
 'SpotWeld': <class 'fibsem.patterning.SpotWeldPattern'>,
 'Trench': <class 'fibsem.patterning.TrenchPattern'>,
 'Undercut': <class 'fibsem.patterning.UndercutPattern'>}


In [4]:
from fibsem.structures import FibsemPatternSettings, FibsemPattern

rect_pattern = patterns_fn["Rectangle"]()

rp = rect_pattern.define(protocol={"pattern": "Rectangle", "width": 10e-6, "height": 5e-6, 
                                              "depth": 2e-6, "rotation": 0, 
                                              "centre_x": 0, "centre_y": 0, 
                                              "scan_direction": "TopToBottom", "cleaning_cross_section": True})


print(rp)

Rectangle
[FibsemPatternSettings(pattern=FibsemPattern.Rectangle, width=1e-05, height=5e-06, depth=2e-06, rotation=0, centre_x=0.0, centre_y=0.0, scan_direction=TopToBottom, cleaning_cross_section=True)]


In [14]:



pattern_fn = patterns_fn["Trench"]()
print(pattern_fn.required_keys)
pattern = pattern_fn.define(protocol=settings.protocol["polish_lamella"])

pprint(pattern)

print("---------------------------------------")

('lamella_width', 'lamella_height', 'trench_height', 'size_ratio', 'offset', 'milling_depth')
[FibsemPatternSettings(pattern=FibsemPattern.Rectangle, width=1.7e-05, height=6e-07, depth=4e-07, rotation=0.0, centre_x=0.0, centre_y=-5.5e-07, scan_direction=BottomToTop, cleaning_cross_section=True),
 FibsemPatternSettings(pattern=FibsemPattern.Rectangle, width=1.7e-05, height=6e-07, depth=4e-07, rotation=0.0, centre_x=0.0, centre_y=5.5e-07, scan_direction=TopToBottom, cleaning_cross_section=True)]
---------------------------------------


In [ ]:
for p in patterns_fn:
    print(p, patterns_fn[p], patterns_fn[p]())

### Test AutoFOCUS



In [2]:
%load_ext autoreload
%autoreload 2

import sys
from fibsem import utils, calibration, patterning
from fibsem.structures import BeamType

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
microscope, settings = utils.setup_session()

2023-03-12 11:47:30,965 — root — INFO — connect_to_microscope:3125 — Connected to Demo Microscope
2023-03-12 11:47:30,967 — root — INFO — setup_session:227 — Finished setup for session: demo_2023-03-12-11-47-30AM


In [6]:
# default

calibration.auto_focus_beam(microscope, settings, beam_type=BeamType.ELECTRON)

2023-03-12 11:47:36,757 — root — INFO — auto_focus:3159 — Auto focus: BeamType.ELECTRON
2023-03-12 11:47:36,759 — root — INFO — auto_focus:3160 — I'm focusing my laser...


In [17]:
# sharpness
from fibsem.calibration import _sharpness, _dog
calibration.auto_focus_beam(microscope, settings, beam_type=BeamType.ELECTRON, metric_fn=_sharpness, kwargs={"disk_size": 5}, num_steps=10)

2023-03-12 11:54:51,255 — root — INFO — get:3325 — Getting working_distance (BeamType.ELECTRON)
2023-03-12 11:54:51,257 — root — INFO — auto_focus_beam:64 — image 0: 3.90e-03
2023-03-12 11:54:51,258 — root — INFO — set:3347 — Setting working_distance to 0.0039000000000000016 (BeamType.ELECTRON)
2023-03-12 11:54:51,263 — root — INFO — autocontrast:3156 — Autocontrast: BeamType.ELECTRON
2023-03-12 11:54:51,273 — root — INFO — _sharpness:105 — calculating sharpness (accutance) of image <fibsem.structures.FibsemImage object at 0x00000174C3EF7220>: 5
2023-03-12 11:54:51,541 — root — INFO — auto_focus_beam:64 — image 1: 3.95e-03
2023-03-12 11:54:51,541 — root — INFO — set:3347 — Setting working_distance to 0.003950000000000001 (BeamType.ELECTRON)
2023-03-12 11:54:51,543 — root — INFO — autocontrast:3156 — Autocontrast: BeamType.ELECTRON
2023-03-12 11:54:51,553 — root — INFO — _sharpness:105 — calculating sharpness (accutance) of image <fibsem.structures.FibsemImage object at 0x00000174C4B05F

In [26]:
calibration.auto_focus_beam(microscope, settings, beam_type=BeamType.ELECTRON, metric_fn=_dog, num_steps=10, verbose=True)

2023-03-12 11:57:44,310 — root — INFO — get:3325 — Getting working_distance (BeamType.ELECTRON)
2023-03-12 11:57:44,311 — root — INFO — auto_focus_beam:50 — _dog based auto-focus routine
2023-03-12 11:57:44,312 — root — INFO — auto_focus_beam:51 — doc: Calculate difference of gaussian (DoG) of an image.

    Args:
        img (FibsemImage): _description_

    Returns:
        _type_: _description_
    
2023-03-12 11:57:44,317 — root — INFO — auto_focus_beam:52 — initial working distance: 2.50e-03
2023-03-12 11:57:44,319 — root — INFO — auto_focus_beam:64 — image 0: 2.25e-03
2023-03-12 11:57:44,320 — root — INFO — set:3347 — Setting working_distance to 0.0022500000000000003 (BeamType.ELECTRON)
2023-03-12 11:57:44,322 — root — INFO — autocontrast:3156 — Autocontrast: BeamType.ELECTRON
2023-03-12 11:57:44,373 — root — INFO — auto_focus_beam:64 — image 1: 2.30e-03
2023-03-12 11:57:44,375 — root — INFO — set:3347 — Setting working_distance to 0.0023000000000000004 (BeamType.ELECTRON)
2023-0